In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

<div class="alert alert-block alert-success">

# EPFL Course: CH-630 Drug Discovery

## Doctoral School EDCH

## Week 4: Exercises

</div>

<h2 style="color:green;"> Lesson 4.4: Enhanced Sampling Methods </h2>

As explained in Lessons 2.3.1 and 2.3.2, **molecular dynamics (MD)** is a computational technique that simulates the time evolution of particles by numerically solving Newton’s equations of motion.

Nowadays, MD can be used to study processes that happens in the microsecond-millisecond scale.
However, several processes of interest in biochemistry and pharmaceutical chemistry, such as protein folding ligand binding and unbinding, happen on way longer scales.
In the context of simulations, these processes are reffered to as **rare events**.

Conventional MD is too slow and computationally expensive to simulate rare events.
More advanced strategies, called **enhanced sampling methods**, allow us to accelerate these events and explore the free energy landscape more efficiently.


In this lesson, we will explore:

1. Setting up Umbrella Sampling simulations

2. Defining bias distributions from Umbrella Sampling simulations

3. Reconstruction of the Potential of Mean Force (PMF)

We will start with a simplified toy model to illustrate the concepts, and later you will move on to a realistic example using GROMACS.

> 💡 This lesson will help you understand how enhanced sampling methods work and how to perform a free energy calculation.

<h2 style="color:green;"> Step 1: Escaping local minima with Umbrella Sampling </h2>

In a real protein-ligand system, the system has thousands to millions of degrees of freedom (all atoms in the protein, ligand, solvent, ions, etc.).
Consequently, the free energy landscape is a $3N$-dimensional hypersurface, where $N$ is the number of atoms in the simulation.

Visualizing such a high-dimensional surface is impossible. Therefore, we usually describe the free energy along a **reaction coordinate** that captures the most important degrees of freedom of the process.
This one-dimensional projection of the free energy landscape is called the **Potential of Mean Force (PMF)**.

For computational simplicity, we will consider again the simple example of a particle moving in a 1D double-well potential.<br><br>

Even though this is a very simplified model, it captures the essential features of a real protein-ligand binding process:

- The lower-energy minimum represents the most stable state, analogous to the bound state where the ligand binds in the protein binding pocket.

- The higher-energy minimum corresponds to the unbound state of the protein-ligand complex.

- The barrier separating the two minima mimics the activation energy between bound and unbound states that needs to be overcome for the process to occur.

In [ ]:
# Define the double-well potential

# Parameters
a = 4.0   # barrier height
b = 1.0   # position of symmetric minima
c = 1.0   # tilt parameter to make one minimum lower

def potential(x, a=a, b=b, c=c):
    """Asymmetric double-well free energy"""
    V = a*(x**2 - b**2)**2 + c*x
    V_shifted = V - np.min(V)
    return V_shifted

x_vals = np.linspace(-1.6, 1.5, 500)
PMF = potential(x_vals)

plt.figure(figsize=(8,5))
plt.plot(x_vals, PMF, 'k-', lw=2)
plt.xlabel('x')
plt.ylabel('Energy')
plt.title('Asymmetric Double-Well')
plt.show()

<div class="alert alert-block alert-warning">
<b>Note:</b><br>

In this simple 1D double-well example, the potential of mean force is known analytically, so we can plot and analyze it directly.

In contrast, in a real protein-ligand system, the shape of the free energy landscape is unknown. We do not know in advance the positions of minima, barriers, or the overall free energy difference. The goal of molecular simulations is to determine this free energy surface by propagating the system along a reaction coordinate (see below) according to Newton’s equations of motion.
</div>

**Binding free energy calculations** often start from a X-ray structure of the protein-ligand complex and simulate unbinding with MD to estimate the free energy surface.

However, with standard MD, the system is often stuck in the lower-energy minimum (bound state), because thermal fluctuations are usually insufficient to cross the barrier within feasible simulation times.

To overcome this limitation, enhanced sampling methods such as **umbrella sampling** are used to accelerate the exploration of rare events and reconstruct the Potential of Mean Force (PMF) along a reaction coordinate.

<h3 style="color:green;"> Step 1.1: Definition of Reaction Coordinate </h3>

A reaction coordinate (RC) is a parameter that describes the progress of a process in a molecular system. It reduces the high-dimensional motion of all atoms to a single, meaningful variable along which we can compute free energies.

In a real protein-ligand system, defining a good reaction coordinate is challenging. Collective varibles (CV) are normally used, which are functions of the atomic coordinates that capture the slowest or most important motions of the system.

CVs can be simple metrics, such as the distance between ligand and protein binding or a torsion angle, or more complex or advanced, such as path collective variables (describing transitions along a predefined path in configuration space) or machine learning-based CVs (using dimensionality reduction or neural networks to identify meaningful collective motions).

The choice of a good reaction coordinate or CV is critical: it should capture the essential physics of the process, allowing enhanced sampling methods to efficiently explore the free energy landscape.

In our 1D double-well example, the reaction coordinate is simply the position of the particle, $x$. This makes it trivial to define and visualize.

<h3 style="color:green;"> Step 1.2: Definition of Umbrella Sampling windows </h3>

In **umbrella sampling** (US) simulations, an additional harmonic biasing potential is applied along the chosen CV $\xi$:

$V_{\rm US}(x) = \frac{1}{2} k (\xi(x)-\xi_0)^2$

where $k$ is the force constant and $\xi_0$ the center of the harmonic restraint.

In our case, the CV is simply the position of the particle, $\xi = x$.


Normally, to reconstruct the PMF along $\xi$ with US, multiple simultions ("windows") are performed, each with the harmonic potential centered at a different value of $\xi$, allowing the system to overcome free energy barriers.<br><br>


Thus, setting up US requires choosing 2 key parameters:
- centers of the umbrella windows $\xi_0$
- the force constant $k$

### Defintion of umbrella centers

In realistic biomolecular simulations, the appropriate values of the umbrella centers are not normally known in advance. Preliminary explorations of the CV can be required to define the number and placement of umbrella sampling windows.

<div class="alert alert-block alert-warning"><b>Note:</b> It is important to ensure that the umbrella windows are placed such that adjacent windows have sufficient overlap in the sampled regions of the CV. Adequate overlap is essential for a reliable reconstruction of the PMF.</div>


However, in our simplified analytical model, the potential is known, and therefore the physically relevant region of the reaction coordinate is also known.

In this simple case, we can use equispaced windows with a distant $\Delta x = 0.5$, from $x=-1.5$ to $x=1.5$.

In [ ]:
# umbrella sampling windows
delta_x = 0.5
centers = np.arange(-1.5, 1.5+delta_x, delta_x)  

### Definition of the force constant

The choice of the harmonic force constant $k$ is also crucial:
- If $k$ is too small, the biasing potential will be weak, and the system may not be able to overcome free energy barriers.
- If $k$ is too large, the system will remain tightly confined around $\xi$, leading to little overlap between windows.

In practice, $k$ should be chosen to balance these two effects and often requires empirical tuning.

Here, we can start from an arbitrary value of $k$ and tune it to have enough overlap between the windows, while crossing the energy barrier.

In [ ]:
# force constant of the harmonic potential
k = 100

In [ ]:
# Visualize the umbrella sampling bias potentials
plt.figure(figsize=(8,5))
for x0_us in centers:
    # umbrella sampling bias potential
    V_bias = 0.5 * k * (x_vals - x0_us)**2
    plt.plot(x_vals, V_bias, label=f'center={x0_us:.2f}')
plt.xlabel('x')
plt.ylabel('Bias potential')
plt.title('Umbrella Sampling Bias Potentials')
plt.ylim(0,7)
plt.legend()
plt.show()

<div class="alert alert-block alert-info">

1. What happens if you increase or reduce the value of $k$?

2. What do you think it is a reasonable value of $k$?
</div>

<h2 style="color:green;"> Step 2: Umbrella Sampling simulations </h2>

Starting from the script of the Velocity Verlet integrator with Andersen thermostat of Lesson 4.3, update it to run US simulations.

In the script for unbias MD: $F(x) = -dV/dx$, where $V$ is the potential of the system. <br><br>

When running umbrella sampling:

$V_{\rm tot}(x) = V(x) + V_{\rm US}(x)$

with $V_{\rm US}(x) = \frac{1}{2} k (x-x_0)^2$<br><br>

Therefore, the total force in an umbrella sampling simulatons is:

$F_{\rm tot}(x) = F(x) + F_{\rm US}(x)$

where $F_{\rm US}(x) = -dV_{\rm US}(x) /dx = - k (x-x_0)$

Modify the function of the force from umbrella sampling according to this latter equation (where indicated). 

In [ ]:
# Define forces for Velocity Verlet integration #

# -------- Double-well force (existing) --------
def force_dw(x, a=a, b=b, c=c):
    """Force from the asymmetric double-well potential F = -dV/dx."""
    return -(4*a*x*(x**2 - b**2) + c)


# -------- Umbrella force --------
def force_umbrella(x, k_us, x0_us):
    """Umbrella force: -k (x - x0)."""
    ### ADD YOUR CODE HERE ###
    F_us = 
    return F_us


# -------- Total force --------
def total_force(x, k_us, x0_us):
    return force_dw(x) + force_umbrella(x, k_us, x0_us)

In [ ]:
# --- Velocity Verlet integrator with Andersen thermostat ---

def velocity_verlet_umbrella(x0, v0, m, dt, n_steps, T=1., nu=0.1, k_us=0, x0_us=0):
    """
    Integrate the motion of a particle using Velocity Verlet with Andersen thermostat (NVT ensemble).
    
    Parameters:
        x0, v0 : initial position and velocity
        m      : mass
        dt     : time step
        n_steps: number of integration steps
        T      : temperature (for thermostat) in reduced units
        nu     : collision frequency for Andersen thermostat
        k_us   : umbrella sampling force constant
        x0_us  : umbrella sampling center position
    Returns:
        x_traj, v_traj, t_traj : arrays of positions, velocities, and times
    """
    kB = 1.0  # Boltzmann constant in reduced units

    x = x0
    v = v0
    F = total_force(x, k_us, x0_us)
    
    x_traj = np.zeros(n_steps)
    v_traj = np.zeros(n_steps)
    t_traj = np.zeros(n_steps)
    
    for i in range(n_steps):
        # --- Velocity Verlet steps ---
        # 1. Update position
        x += v * dt + 0.5 * F/m * dt**2 
        # 2. Compute new force
        F_new = total_force(x, k_us, x0_us) 
        # 3. Update velocity
        v += 0.5 * (F + F_new)/m * dt 
        # 4. Update force for next step
        F = F_new
        
        # --- Andersen thermostat ---
        if np.random.rand() < nu * dt:
            # Reassign velocity from Maxwell-Boltzmann distribution
            v = np.random.normal(0, np.sqrt(kB * T / m))

        # Store trajectories
        x_traj[i] = x
        v_traj[i] = v
        t_traj[i] = i*dt
        
    return t_traj, x_traj, v_traj

Now, we can run the umbrella sampling simulations in the NVT ensemble, each with the bias harmonic potential centered in $x_0$ from -1.5 to 1.5.

In [ ]:
# number of steps of the simulation (the longer, the better sampling)
n_steps = 5000000

# store trajectory for each umbrella center
trajectories = np.zeros((len(centers), n_steps))


# Run US simulations for each umbrella center
for i,x0_us in enumerate(centers):
    print(f"Running umbrella sampling simulation for center at x={x0_us:.2f}")
    t, x, v = velocity_verlet_umbrella(
    x0=x0_us,         # start the simulation in the center of the umbrella
    v0=0.0,
    m=1.0,
    dt=0.001,
    n_steps=n_steps,
    T=1,              # reduced temperature
    nu=0.1,
    k_us=100,         # umbrella strength
    x0_us=x0_us       # umbrella center
    )

    trajectories[i] = x  # Store the trajectory for analysis


In [ ]:
# Plot histogram for each umbrella window
for i, x0_us in enumerate(centers):
    plt.hist(trajectories[i],bins=20,density=True,alpha=0.5,label=f"Center = {x0_us:.2f}")

plt.xlabel("x coordinate")
plt.ylabel("Biased probability density")
plt.title("Biased Histograms from Umbrella Sampling Windows")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

This plot show the histograms of the coordinate sampled during each US simulations.

<div class="alert alert-block alert-info">

1. Do you have enough overlap between subsequent umbrella distributions?

2. Try increasing or decreasing the $k$ to see how it affects the histograms.

</div>

You might have noticee that with $k=100$, the distributions do not overlap.
- If $k$ is too high, the overlap becomes very poor.
- If $k$ is too small, it becomes very hard to sample around high-energy regions, and the distributions will be mostly confined to the minima.

<div class="alert alert-block alert-info">

Which range of $k$ values gives you a good overlap between adjacent distributions?

</div>

(Hint: for the next step, you will need a good overlap and good sampling.

Try $k=20$ and increase `n_steps` to at least 2000000 - this will take some minutes.)

<h2 style="color:green;"> Step 3: Reconstruction of the Potential of Mean Force (PMF) </h2>

The potential of mean force along a reaction coordinate $x$ can be obtain from the **Boltzmann distributions** of $x$ in the NVT ensemble:

$\rm PMF(x) = - k_B*T * \ln \rho(x)$

where $\rho(x)$ is the probability density of observing the system at position $x$.<br><br>

However, the histograms we collected from umbrella sampling correspond to **biased, non-Boltzmann distribributions**.

This is because we sampled $x$ accoring not only to the potential of the system, $V(x)$, but also the bias umbrella sampling potenital $V_{\rm US}(x)$.<br><br>

To recover the unbias Boltzmann distribution and determine the PMF, we need to correct for this bias. This is exactly what post-processing methods like WHAM (Weighted Histogram Analysis Method) do.

In [ ]:
# Download WHAM release 2.1.0 from the Grossfield lab http://membrane.urmc.rochester.edu/?page_id=79
!wget http://membrane.urmc.rochester.edu/sites/default/files/wham/wham-release-2.1.0.tgz

# Extract the tarball
!tar -xvzf wham-release-2.1.0.tgz

# install WHAM
!mkdir wham/build
!cd  wham/build && cmake .. &&cmake --build .

In [ ]:
# You might need to change permissions to make the wham executable runnable
#!chmod +x wham/wham

To use WHAM, we need to prepare a file for each umbrella sampling window contaning two colums: time and $x$ values sampled during US.

In [ ]:
# Create a folder to store the umbrella data
os.makedirs("umbrella_sampling", exist_ok=True)

# Time step and number of steps
dt=0.001
n_steps = trajectories.shape[1]

# Loop over windows and save files
for i, x0_us in enumerate(centers):
    t = np.arange(n_steps) * dt
    x = trajectories[i]
    data = np.column_stack((t, x))
    
    filename = f"umbrella_sampling/window_{x0_us}.dat"
    np.savetxt(filename, data, fmt="%.6f", comments='', delimiter=' ')

Then, we need to define a metadata file, which lists for each umbrella window:
- the corresponding file name
- the umbrella center $x_0$ 
- the force constant $k$.

In [ ]:
# WHAM PMF is calculated from different umbrella windows
file_names = [f"umbrella_sampling/window_{x0_us}.dat" for x0_us in centers]
k_us =  # umbrella force constant in reduced units

# Prepare WHAM input file
wham_file = pd.DataFrame({
    'file names': file_names,
    'x_0': centers,
    'k': [k_us]*len(centers)
})

# Save the metadata file for WHAM
wham_file.to_csv(f'umbrella_sampling/wham_us', index=False, header=False, sep=' ')

Now, we can run WHAM by specifying:
- the units (here, reduced units with $k_B=1$)
- the minimum and maximum of the histogram
- the number of bins
- the tolerance for convergence
- the temperature (in reduced units, T=1)
- the perodicity of the coordinate (0 because the coordinate is not periodic)
- the name of the metadata file
- the name of the output PMF file

In [ ]:
!  wham/build/wham units lj -1.6 1.5 80 0.0001 1 0 umbrella_sampling/wham_us umbrella_sampling/pmf_us

Finally, we can compare the Potential of Mean Force that we obtained from Umbrella Sampling simulations and WHAM with the origial potential.

(Note: we need to convert the WHAM PMF from kcal/mol to reduced units)

In [ ]:
# read the pmf results
pmf_us = pd.read_csv(f'umbrella_sampling/pmf_us', comment='#', header=None, sep='\t')

# plot the WHAM PMF against the true PMF
plt.plot(pmf_us[0], pmf_us[1], label='WHAM PMF', color='blue', lw=2)
plt.plot(x_vals, PMF, 'k-', lw=2, label='True PMF')
plt.xlabel('x', fontsize=15, font='Helvetica', labelpad=12)
plt.ylabel('PMF', fontsize=15, font='Helvetica', labelpad=12)
plt.legend()
plt.grid()
plt.show()

If you started from a good overlap between adjecent window and sampled enough, you should see that the PMF determined with umbrella sampling matches the original function.

(Small differences might be due to an incomplete sampling - if there are differences, you can try increasing the number of steps of the US simulations)

<div class="alert alert-block alert-warning">

This is particularly important in free energy calculations, because, unlike this toy model, we generally do not know the shape of the free energy landscape.

Therefore, enhanced sampling methods like umbrella sampling are invaluable for estimating the PMF of biological processes, such as protein-ligand binding.

This allows us to calculate the binding free energy of a protein-ligand complex, which is crucial in drug discovery, as it reflects the affinity of a ligand, hence a potential drug, for its protein target.

</div>

<h2 style="color:orange;"> Exercise </h2>

Now that you have explored umbrella sampling using a simple one-dimensional model, it is time to move to a realistic biomolecular system. <br><br>

In this final exercise, you will run an all-atom MD umbrella sampling simulation using GROMACS.

You will follow the tutorial at https://tutorials.gromacs.org/umbrella-sampling.html (doi:10.5281/zenodo.11198375)

(You can also follow the tutorial online at https://mybinder.org/v2/gl/gromacs%2Fonline-tutorials%2Fumbrella-sampling/main?filepath=tutorial.ipynb)